In [9]:
import pandas as pd
pd.options.mode.chained_assignment = None
import numpy as np
import os
import random
from scipy import stats
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

In [10]:
# load prepared data
tdata = pd.read_csv('raw_data/merged_data.csv')

In [14]:
# # excluding non-European participants
# mapping_IDs = pd.read_csv('raw_data/mapping_IDs.csv')
# EUR_subjs = pd.read_csv('raw_data/ukb_European_PRSs.csv')
# mapping_IDs = mapping_IDs[mapping_IDs['karin_id'].isin(EUR_subjs['FID'])]
# tdata = tdata[tdata['eid'].isin(mapping_IDs['guido_id'])]

In [3]:
# select CSA and CT data from prepared data
area_items = pd.read_csv('raw_data/DK_CSA_items.csv')
s1 = ['eid']
for i in range(area_items.shape[0]):
    s1.append(str(area_items.iloc[i,0])+'-2.0')
CSA_data = tdata[s1].set_index('eid')

thickness_items = pd.read_csv('raw_data/DK_CT_items.csv')
s2= ['eid']
for i in range(thickness_items.shape[0]):
    s2.append(str(thickness_items.iloc[i,0])+'-2.0')
CT_data = tdata[s2].set_index('eid')

In [4]:
#load ALFF and ReHo data
ALFF_data = pd.read_csv('raw_data/ALFF_DK_atlas.csv')
ReHo_data = pd.read_csv('raw_data/ReHo_DK_atlas.csv')
ALFF_data = ALFF_data.set_index('eid')
ReHo_data = ReHo_data.set_index('eid')

#match functional and structural brain data
tmp_l = list(set(ALFF_data.index) & set(CT_data.index))
CSA_data = CSA_data.loc[tmp_l]
CT_data = CT_data.loc[tmp_l]

ALFF_data = ALFF_data.loc[tmp_l]
regions = area_items.iloc[:,1].values.tolist()
ALFF_data = ALFF_data[regions]

ReHo_data = ReHo_data.loc[tmp_l]
regions = area_items.iloc[:,1].values.tolist()
ReHo_data = ReHo_data[regions]

In [5]:
# extract covariates from prepared data
df = pd.DataFrame({"eid" : tdata['eid']})
df['Age'] = tdata['21003-2.0']
df['Sex'] = tdata['31-0.0']
df['IQ'] = tdata['20016-2.0']
df['EA'] = tdata['6138-2.0']
df['motion'] = tdata['25741-2.0']
df['TCSA'] = tdata['26721-2.0'] + tdata['26822-2.0']
df['ACT'] = tdata['26755-2.0'] + tdata['26856-2.0']
df.set_index('eid',inplace=True)
df = df.loc[tmp_l].reset_index()

In [6]:
# print demographic information
print("sample size:",df.shape[0])
age = df['Age'].values
print("min age:", np.min(age),"max age:",np.max(age),"mean age:", np.mean(age),"std age:",np.std(age))
sex = df['Sex'].values
print("females:",np.sum(sex==0),"males:",np.sum(sex==1))

sample size: 19205
min age: 45.0 max age: 81.0 mean age: 62.9747982296277 std age: 7.4390828237902165
females: 9699 males: 9506


In [7]:
# select risky behaviors
s = ['eid','2040-2.0','1100-2.0','20160-2.0','Alc','2149-2.0']
data1 = tdata[s]

In [8]:
# transfer Alchol drinks per week and sexual paterners to distinct values 
# Alcohol: [0,5]-> 1 ; (5,10] -> 2; (10,15] -> 3; (15,20] -> 4; >20 -> 5 
# Sexual parteners: 1-1, (1,5]-> 2, (5,10]-3, (10,20]-4, >=20 - 5
data1.set_index('eid',inplace=True)
Y = data1.loc[tmp_l].values
new_Alc = []
new_Sex = []
for i in range(Y.shape[0]):
    if Y[i,-1] == 1:
        new_Sex.append(1)
    if Y[i,-1] >= 2 and Y[i,-1] <= 5:
        new_Sex.append(2)
    if Y[i,-1] > 5 and Y[i,-1] <= 10:
        new_Sex.append(3)
    if Y[i,-1] > 10 and Y[i,-1] <= 20:
        new_Sex.append(4)
    if Y[i,-1] > 20:
        new_Sex.append(5)
    
    if Y[i,-2] <= 5:
        new_Alc.append(1)
    if Y[i,-2] > 5 and Y[i,-2] <= 10:
        new_Alc.append(2)
    if Y[i,-2] > 10 and Y[i,-2] <= 15:
        new_Alc.append(3)
    if Y[i,-2] > 15 and Y[i,-2] <= 20:
        new_Alc.append(4)
    if Y[i,-2] > 20:
        new_Alc.append(5)

RT_data = data1.loc[tmp_l]
RT_data['2149-2.0'] = new_Sex
RT_data['Alc'] = new_Alc

In [9]:
# function to conduct the regression of covariates

def regression_covariant(covariant_matrix, y, standard_scale=True):
    a = np.hstack((covariant_matrix,np.ones((covariant_matrix.shape[0], 1))))
    w = np.linalg.lstsq(a,y,rcond=None)[0]

    residual = y - covariant_matrix.dot(w[:-1])
    residual = residual.astype('float64')

    if standard_scale:
        residual = StandardScaler().fit_transform(residual.reshape(-1,1)).flatten()

    return residual, w

def regressing_data(imaging_data,df,imaging_type):
    co_cols = ['Age','Sex','IQ','EA']
#     co_cols = ['Age','Sex']
    if imaging_type == 'CSA':
        co_cols.append('TCSA')
    
    if imaging_type == 'CT':
        co_cols.append('ACT')
    
    if imaging_type == 'ALFF' or imaging_type == 'ReHo':
        co_cols.append('motion')
    
    co = df[co_cols].values
    s = imaging_data.shape
    reg_imaging_data = np.zeros(s)
    for i in range(s[1]):
        x = imaging_data.iloc[:,i].values
        rx,w = regression_covariant(co,x,standard_scale=False)
        reg_imaging_data[:,i] = rx
    
    return reg_imaging_data

In [12]:
# Use PCA to extract the PCs of four risky behaviors
# normalization
Y_tmp = stats.zscore(RT_data.values[:,1:],axis=0)
# PCA analysis
pca = PCA(n_components=4)
pca_exp = pca.fit(Y_tmp)
Y_c = stats.zscore(pca_exp.transform(Y_tmp),axis=0)
print("explained variance ratio of PCs for risky behaviors:",pca_exp.explained_variance_ratio_)

#correlations between PC1 and four risky behaviors
for i in range(4):
    print(s[i+2],stats.spearmanr(Y_c[:,0],Y_tmp[:,i]))
df_PCs = pd.DataFrame(data=Y_c,columns=['PC1','PC2','PC3','PC4'])
df_PCs.to_csv('results/imaging_associations/PCs_risky_behaviors.csv',index=False)

explained variance ratio of PCs for risky behaviors: [0.36578415 0.23404963 0.20662033 0.1935459 ]
1100-2.0 SpearmanrResult(correlation=0.5298986816070743, pvalue=0.0)
20160-2.0 SpearmanrResult(correlation=0.595717677724391, pvalue=0.0)
Alc SpearmanrResult(correlation=0.575358717978464, pvalue=0.0)
2149-2.0 SpearmanrResult(correlation=0.6466111287997693, pvalue=0.0)


In [13]:
# From the covariance matrix, caluclate the Eigenvalues and the Eigenvectors
cov_mat = np.cov(Y_tmp.T)
eigen_vals, eigen_vecs = np.linalg.eig(cov_mat)
tmp = pd.DataFrame(np.vstack((eigen_vecs,eigen_vals,pca_exp.explained_variance_ratio_)),columns=['PC1','PC2','PC3','PC4'])
tmp.to_csv('results/imaging_associations/eigen_vals_and_vecs.csv',index=False)

In [14]:
# regressing out covariates
reg_CSA_data = regressing_data(CSA_data,df,'CSA')
reg_CT_data = regressing_data(CT_data,df,'CT')
reg_ALFF_data = regressing_data(ALFF_data,df,'ALFF')
reg_ReHo_data = regressing_data(ReHo_data,df,'ReHo')
reg_df_PCs1 = regressing_data(df_PCs,df,'CSA')
reg_df_PCs2 = regressing_data(df_PCs,df,'CT')
reg_df_PCs3 = regressing_data(df_PCs,df,'ALFF')
reg_df_PCs4 = regressing_data(df_PCs,df,'ReHo')

In [15]:
# create empty array for correlations
CSA_corr = np.zeros((len(regions),3))
CT_corr = np.zeros((len(regions),3))
ALFF_corr = np.zeros((len(regions),3))
ReHo_corr = np.zeros((len(regions),3))

In [16]:
# FDR multiple comparison correction
from statsmodels.stats import multitest
def fdr_correction(data):
    Ps = multitest.multipletests(data,alpha=0.05,method='fdr_bh')
    return Ps[1]

/data/sliu/anaconda3/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [17]:
# Calculate the correlations between PCs of risky behaviors and brain measures
for PC in range(4):
    for j in range(len(regions)):
        x1 = reg_CSA_data[:,j]
        r,p = stats.spearmanr(x1,reg_df_PCs1[:,PC])
        CSA_corr[j,0] = r
        CSA_corr[j,1] = p
    CSA_corr[:,2] = fdr_correction(CSA_corr[:,1])

    for j in range(len(regions)):
        x1 = reg_CT_data[:,j]
        r,p = stats.spearmanr(x1,reg_df_PCs2[:,PC])
        CT_corr[j,0] = r
        CT_corr[j,1] = p  
    CT_corr[:,2] = fdr_correction(CT_corr[:,1])

    for j in range(len(regions)):
        x1 = reg_ALFF_data[:,j]
        r,p = stats.spearmanr(x1,reg_df_PCs3[:,PC])
        ALFF_corr[j,0] = r
        ALFF_corr[j,1] = p  
    ALFF_corr[:,2] = fdr_correction(ALFF_corr[:,1])

    for j in range(len(regions)):
        x1 = reg_ReHo_data[:,j]
        r,p = stats.spearmanr(x1,reg_df_PCs4[:,PC])
        ReHo_corr[j,0] = r
        ReHo_corr[j,1] = p  
    ReHo_corr[:,2] = fdr_correction(ReHo_corr[:,1])

    # save correlations
    df1 = pd.DataFrame(data=CSA_corr,columns=['r','p','fdr_p'])
    df1['regions'] = regions
    df1.to_csv('results/imaging_associations/CSA_corr_PC'+str(PC+1)+'.csv',index=False)
    df2 = pd.DataFrame(data=CT_corr,columns=['r','p','fdr_p'])
    df2['regions'] = regions
    df2.to_csv('results/imaging_associations/CT_corr_PC'+str(PC+1)+'.csv',index=False)
    df3 = pd.DataFrame(data=ALFF_corr,columns=['r','p','fdr_p'])
    df3['regions'] = regions
    df3.to_csv('results/imaging_associations/ALFF_corr_PC'+str(PC+1)+'.csv',index=False)
    df4 = pd.DataFrame(data=ReHo_corr,columns=['r','p','fdr_p'])
    df4['regions'] = regions
    df4.to_csv('results/imaging_associations/ReHo_corr_PC'+str(PC+1)+'.csv',index=False)

In [19]:
# produce the funcPC1 based on brain associations of ALFF and ReHo
df1 = pd.read_csv('results/imaging_associations/ALFF_corr_PC1.csv')
df2 = pd.read_csv('results/imaging_associations/ReHo_corr_PC1.csv')
func_corr = np.vstack((df1['r'].values,df2['r'].values)).T

Y_tmp = stats.zscore(func_corr,axis=0)
pca = PCA(n_components=2)
pca_exp = pca.fit(Y_tmp)
Y_c = stats.zscore(pca_exp.transform(Y_tmp),axis=0)
print("explained variance ratio of PCs for functional brain associations:",pca_exp.explained_variance_ratio_)

df_PCs = pd.DataFrame(data=Y_c,columns=['PC1','PC2'])
df_PCs['regions'] = regions
df_PCs.to_csv('results/imaging_associations/func_PCs.csv',index=False)

explained variance ratio of PCs for functional brain associations: [0.83635846 0.16364154]


In [20]:
print("associations between ALFF and ReHo:",stats.spearmanr(df1['r'].values,df2['r'].values))

associations between ALFF and ReHo: SpearmanrResult(correlation=0.6580732700135685, pvalue=1.922295591643946e-09)
